In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from AbsPerfectPower import AbsPerfectPower as appwr 
#-----
from DOVSAudit import DOVSAudit
from DABatch import DABatch
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [ ]:
# TODO: WHAT TO DO FOR ENTRIES WITH PN = NONE?

In [ ]:
opcos = ['ap', 'ky', 'oh', 'im', 'pso', 'swp', 'tx']

In [ ]:
conn_aws  = Utilities.get_athena_prod_aws_connection()

In [ ]:
save_dir = r'C:\Users\s346557\Documents\LocalData\abs_perfect_power'
#-----
date_0   = pd.to_datetime('2024-11-11')
date_1   = pd.to_datetime('2024-11-17')

# date_0 = pd.to_datetime('2024-01-01')
# date_1 = pd.to_datetime('2024-09-30')
#-----
save_subdir = f"{date_0.strftime('%Y%m%d')}_{date_1.strftime('%Y%m%d')}"
#-----
n_max = int((date_1-date_0+pd.Timedelta('1day'))/pd.Timedelta('15min'))

In [ ]:
sql_v1F = appwr.build_zero_times_sql_v1(
    date_0           = date_0,
    date_1           = date_1,
    split_if_missing = False, 
    opco             = 'oh'
)

sql_v1T = appwr.build_zero_times_sql_v1(
    date_0           = date_0,
    date_1           = date_1,
    split_if_missing = True, 
    opco             = 'oh'
)

sql_v2 = appwr.build_zero_times_sql_v2(
    date_0           = date_0,
    date_1           = date_1,
    opco             = 'oh'
)

In [ ]:
start = time.time()

df_v1F = pd.read_sql_query(
    sql_v1F, 
    conn_aws
)
print(time.time()-start)

In [ ]:
start = time.time()

df_v1T = pd.read_sql_query(
    sql_v1T, 
    conn_aws
)
print(time.time()-start)

In [ ]:
start = time.time()

df_v2 = pd.read_sql_query(
    sql_v2, 
    conn_aws
)
print(time.time()-start)

In [ ]:
df_v1F.head()

In [ ]:
df_v1T.head()

In [ ]:
df_v2.head()

In [ ]:
print(df_v1F.shape)

In [ ]:
print(df_v1T.shape)
print(df_v2.shape)

In [ ]:
cols_to_comp = ['serialnumber', 'island_start_starttimeperiod', 'island_end_starttimeperiod']

In [ ]:
df_v1T[cols_to_comp].sort_values(by=cols_to_comp, ignore_index=True).equals(
df_v2[cols_to_comp].sort_values(by=cols_to_comp, ignore_index=True)
)

In [ ]:
df_v2['serialnumber'].value_counts()

In [ ]:
vcs_2 = df_v2['serialnumber'].value_counts()
vcs_2

In [ ]:
fig,ax = Plot_General.default_subplots()
sns.histplot(ax=ax, data=vcs_2, bins=10, stat='count')

In [ ]:
import AbsPerfectPower
reload(AbsPerfectPower)
from AbsPerfectPower import AbsPerfectPower as appwr

In [ ]:
fig,ax = Plot_General.default_subplots()
sns.histplot(ax=ax, data=vcs_2.clip(upper=25), bins=[0,1,7,13,25,26], stat='count')
ax.set_yscale('log')
ax.bar_label(ax.containers[0], fmt='%.0f');

In [ ]:
# import AbsPerfectPower
# reload(AbsPerfectPower)
# from AbsPerfectPower import AbsPerfectPower as appwr

In [ ]:
figax =  appwr.histplot_acc_srvc_intrrptns(
    zero_times_df = df_v2, 
    bins                  = None, 
    supplmntl_data        = None, 
    stat                  = 'count', 
    figax                 = None, 
    fig_num               = 0, 
    set_logy              = True, 
    title                 = None, 
    include_text          = True, 
    #-----
    SN_col                = 'serialnumber', 
)

In [ ]:
figax =  appwr.histplot_acc_srvc_intrrptns(
    zero_times_df = df_v2, 
    bins                  = None, 
    supplmntl_data        = None, 
    stat                  = 'probability', 
    figax                 = None, 
    fig_num               = 0, 
    set_logy              = True, 
    title                 = None, 
    include_text          = True, 
    #-----
    SN_col                = 'serialnumber', 
)

In [ ]:
save_dir = r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\OPCOs_v1T'
#-----
date_0   = pd.to_datetime('2023-10-01')
date_1   = pd.to_datetime('2024-09-30')
daq_freq = 'MS'
#-----

In [ ]:
conn_aws  = Utilities.get_athena_prod_aws_connection()

In [ ]:
# opcos = ['ap', 'ky', 'oh', 'im', 'pso', 'swp', 'tx']
opcos = ['ap', 'oh', 'im', 'pso', 'swp']
# opcos = ['oh']

In [ ]:
appwr.run_daq(
    save_dir         = save_dir, 
    date_0           = date_0, 
    date_1           = date_1, 
    opcos            = opcos, 
    daq_freq         = daq_freq, 
    conn_aws         = conn_aws, 
    allow_null_SNs   = True, 
    allow_null_PNs   = True, 
    split_if_missing = True, 
    method           = 'v2'
)

In [ ]:
assert(0)

In [ ]:
for opco_i in opcos:
    for dates_i in dates:
        date_i_0 = dates_i[0].strftime('%Y-%m-%d')
        date_i_1 = dates_i[1].strftime('%Y-%m-%d')
        #-------------------------
        print(f"opco = {opco_i}\ndate_0 = {date_i_0}\ndate_1 = {date_i_1}\n")
        #-------------------------
        save_dir_i = os.path.join(save_dir, opco_i)
        if not os.path.exists(save_dir_i):
            os.makedirs(save_dir_i)
        #-------------------------
        sql_i = appwr.build_app_zero_times_sql_v1(
            date_0           = date_i_0,
            date_1           = date_i_1,
            split_if_missing = True, 
            opco             = opco_i
        )
        df_i = pd.read_sql_query(
            sql_i, 
            conn_aws
        )
        #-------------------------
        save_name_i = date_i_0.replace('-','') + '_' + date_i_1.replace('-','') + '.pkl'
        df_i.to_pickle(os.path.join(save_dir_i, save_name_i))

In [ ]:
# import AbsPerfectPower
# reload(AbsPerfectPower)
# from AbsPerfectPower import AbsPerfectPower as appwr

In [ ]:
opco_dfs = {}
for opco_i in opcos:
    save_dir_i = os.path.join(save_dir, opco_i)
    df_i = Utilities_df.concat_dfs_in_dir(
        dir_path             = save_dir_i, 
        regex_pattern        = r'\d{8}_\d{8}.*', 
        ignore_case          = False, 
        ext                  = '.pkl', 
        make_col_types_equal = False, 
        return_paths         = False
    )
    opco_dfs[opco_i] = df_i


In [ ]:
opco_dfs['ap']

In [ ]:
table_dfs = {}
for opco_i in opcos:
    table_dfs[opco_i] = appwr.build_acc_srvc_intrrptns_table_df(
        zero_times_df = opco_dfs[opco_i], 
        supplmntl_data        = None
    )

In [ ]:
table_dfs['ap']

In [ ]:
app_SNs_oh = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_oh.pkl')
perfect_SNs = list(set(app_SNs_oh['serialnumber'].unique()).difference(set(dev_df_i['serialnumber'].unique())))

In [ ]:
app_SNs_ap  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_ap.pkl')
app_SNs_im  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_im.pkl')
app_SNs_oh  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_oh.pkl')
app_SNs_pso = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_pso.pkl')
app_SNs_swp = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_swp.pkl')

In [ ]:
opco_n_perfect_SNs = dict()
opco_n_perfect_SNs['ap']  = len(list(set(app_SNs_ap['serialnumber'].unique()).difference(set(opco_dfs['ap']['serialnumber'].unique()))))
opco_n_perfect_SNs['im']  = len(list(set(app_SNs_im['serialnumber'].unique()).difference(set(opco_dfs['im']['serialnumber'].unique()))))
opco_n_perfect_SNs['oh']  = len(list(set(app_SNs_oh['serialnumber'].unique()).difference(set(opco_dfs['oh']['serialnumber'].unique()))))
opco_n_perfect_SNs['pso'] = len(list(set(app_SNs_pso['serialnumber'].unique()).difference(set(opco_dfs['pso']['serialnumber'].unique()))))
opco_n_perfect_SNs['swp'] = len(list(set(app_SNs_swp['serialnumber'].unique()).difference(set(opco_dfs['swp']['serialnumber'].unique()))))

In [ ]:
idk = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\ap\20231001_20231031\METHOD_1\app_df_agg_ap.pkl')

In [ ]:
idk['serialnumber'].nunique()

In [ ]:
app_SNs_ap['serialnumber'].nunique()

In [ ]:
acc_srvc_intrrptns_dict = {}
for opco_i in opcos:
    vcs_srs_i       = opco_dfs[opco_i]['serialnumber'].value_counts()
    n_perfect_SNs_i = opco_n_perfect_SNs[opco_i]
    acc_srvc_intrrptns_dict[opco_i] = appwr.build_acc_srvc_intrrptns_table_df(
        acc_srvc_intrrptns_df = opco_dfs[opco_i], 
        supplmntl_data        = {0: opco_n_perfect_SNs[opco_i]}
    )

In [ ]:
acc_outg_drtn_dict = {}
for opco_i in opcos:
    df_i = opco_dfs[opco_i].copy()
    n_perfect_SNs_i = opco_n_perfect_SNs[opco_i]
    #-----
    df_i['delta'] = pd.to_datetime(df_i['island_end_starttimeperiod'])-pd.to_datetime(df_i['island_start_starttimeperiod'])+pd.Timedelta('15min')
    df_i['n_15T'] = df_i['delta']/pd.Timedelta('15min')
    #-----
    df_i = df_i.groupby(['serialnumber'])['delta'].sum()
    df_i = df_i.div(pd.Timedelta('1 hour'))

    df_i = appwr.build_table_df(
        n_intr_srs     = df_i.copy(), 
        bins           = [
            (None, 6), 
            (6,24), 
            (24,48), 
            (48,96), 
            (96, None)
        ], 
        supplmntl_data = {(None, 6): n_perfect_SNs_i}
    )
    acc_outg_drtn_dict[opco_i] = df_i

In [ ]:
acc_srvc_intrrptns_dict['oh']

In [ ]:
acc_outg_drtn_dict['oh']

In [ ]:
assert(acc_srvc_intrrptns_dict.keys()==acc_outg_drtn_dict.keys())
opcos = list(acc_srvc_intrrptns_dict.keys())

In [ ]:
acc_srvc_intrrptns_dict['oh']

In [ ]:
figax =  appwr.histplot_acc_srvc_intrrptns(
    acc_srvc_intrrptns_df = acc_srvc_intrrptns_dict['oh'], 
    bins                  = None, 
    supplmntl_data        = None, 
    stat                  = 'probability', 
    figax                 = None, 
    fig_num               = 0, 
    set_logy              = True, 
    title                 = None, 
    include_text          = True, 
    #-----
    SN_col                = 'serialnumbers', 
)

In [ ]:
startcol_0 = 0
startcol_1 = 5
startrow_i = 0

with pd.ExcelWriter(r'C:\Users\s346557\Downloads\Test.xlsx') as writer:  
    for opco_i in opcos:
        srvc_intrrptns_i = acc_srvc_intrrptns_dict[opco_i].copy()
        outg_drtns_i     = acc_outg_drtn_dict[opco_i].copy()
        #-----
        srvc_intrrptns_i = Utilities_df.prepend_level_to_MultiIndex(df=srvc_intrrptns_i, level_val=f"Acc. Service Interruptions: OPCO = {opco_i}", axis=1)
        outg_drtns_i     = Utilities_df.prepend_level_to_MultiIndex(df=outg_drtns_i, level_val=f"Acc. Outage Duration: OPCO = {opco_i}", axis=1)
        #-----
        srvc_intrrptns_i.to_excel(writer, startrow=startrow_i, startcol=startcol_0)
        outg_drtns_i.to_excel(     writer, startrow=startrow_i, startcol=startcol_1)
        startrow_i += np.max([srvc_intrrptns_i.shape[0], outg_drtns_i.shape[0]])+5

In [ ]:
opco_i='oh'

In [ ]:
df_i = opco_dfs[opco_i].copy()
n_perfect_SNs_i = opco_n_perfect_SNs[opco_i]
#-----
df_i['delta'] = pd.to_datetime(df_i['island_end_starttimeperiod'])-pd.to_datetime(df_i['island_start_starttimeperiod'])+pd.Timedelta('15min')
df_i['n_15T'] = df_i['delta']/pd.Timedelta('15min')
#-----
df_i = df_i.groupby(['serialnumber'])['delta'].sum()
df_i = df_i.div(pd.Timedelta('1 hour'))

In [ ]:
df_i

In [ ]:
dev_df_i = opco_dfs['oh'].copy()
dev_df_i

In [ ]:
dev_df_i['delta'] = pd.to_datetime(dev_df_i['island_end_starttimeperiod'])-pd.to_datetime(dev_df_i['island_start_starttimeperiod'])+pd.Timedelta('15min')
dev_df_i['n_15T'] = dev_df_i['delta']/pd.Timedelta('15min')

In [ ]:
dev_df_i2 = dev_df_i.groupby(['serialnumber'])['delta'].sum()
dev_df_i2 = dev_df_i2.div(pd.Timedelta('1 hour'))
dev_df_i2

In [ ]:
dev_df_i3 = appwr.build_table_df(
    n_intr_srs = dev_df_i2.copy(), 
    bins       = [
        (None, 6), 
        (6,24), 
        (24,48), 
        (48,96), 
        (96, None)
    ]
)

In [ ]:
dev_df_i4 = appwr.build_table_df(
    n_intr_srs     = dev_df_i2.copy(), 
    bins           = [
        (None, 6), 
        (6,24), 
        (24,48), 
        (48,96), 
        (96, None)
    ], 
    supplmntl_data = {(None, 6): len(perfect_SNs)}
)

In [ ]:
dev_df_i3

In [ ]:
dev_df_i4

In [ ]:
n_total_ap = get_nz_nt_ratio_partial_df(
    date_0       = '2023-10-01', 
    date_1       = '2024-09-30', 
    measure      = 'n_total', 
    method       = 'v2', 
    conn_aws     = conn_aws, 
    opcos        = ['ap']
)

In [ ]:
n_total_ap

In [ ]:
# df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\ap\20231001_20231031\METHOD_1\app_df_agg_ap.pkl')
# df_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\ap\20231001_20231031\METHOD_1\app_SNs_ap.pkl')

df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\oh\20231001_20231031\METHOD_1\app_df_agg_oh.pkl')
df_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\oh\20231001_20231031\METHOD_1\app_SNs_oh.pkl')

In [ ]:
print(df_1.shape)
print(df_2.shape)

In [ ]:
df_1.head()

In [ ]:
df_2.head()

In [ ]:
print(df_1['serialnumber'].nunique())
print(df_2['serialnumber'].nunique())

In [ ]:
dffs = list(set(df_1['serialnumber'].unique()).symmetric_difference(set(df_2['serialnumber'].unique())))
len(dffs)

In [ ]:
ugh_0 = """
SELECT
	aep_premise_nb,
	serialnumber,
	COUNT(DISTINCT starttimeperiod)
FROM usage_nonvee.reading_ivl_nonvee
WHERE aep_opco = 'oh'
AND   aep_usage_dt BETWEEN '2023-10-01' AND '2023-10-31'
AND   aep_derived_uom = 'VOLT'
GROUP BY
	aep_premise_nb,
	serialnumber
"""

In [ ]:
df_ugh_0 = pd.read_sql_query(
    ugh_0, 
    conn_aws
)

In [ ]:
df_ugh['serialnumber'].nunique()

In [ ]:
ugh = """
SELECT
	serialnumber,
	COUNT(DISTINCT starttimeperiod)
FROM usage_nonvee.reading_ivl_nonvee
WHERE aep_opco = 'oh'
AND   aep_usage_dt BETWEEN '2023-10-01' AND '2023-10-31'
AND   aep_derived_uom = 'VOLT'
GROUP BY
	serialnumber
"""

In [ ]:
df_ugh = pd.read_sql_query(
    ugh, 
    conn_aws
)

In [ ]:
df_ugh['serialnumber'].nunique()

In [ ]:
df_ugh

In [ ]:
amanda_PNs_UTL_sql = """
with prem as (
	select
		a.prem_nb,
		a.co_cd_ownr,
		trim(a.serv_city_ad) as serv_city_ad,
		a.state_cd,
		a.serv_zip_ad,
		a.dvsn_cd,
		a.area_cd,
		a.acct_clas_cd, -- Make sure grab the latest bill account
		row_number() over (partition by a.prem_nb order by a.turn_on_dt desc) as rn
	from UTLUSER.ods_mcs_indicativ_data a
	 where exists (
		select 1
	      from utluser.ods_mds_installed_devc x
		where x.mtr_kind_cd = 'K'
	   and a.acct_type_cd IN ('E', 'EO', 'AC')
	   and ( x.rmvl_ts is null or x.rmvl_ts >= cast('2023-10-27 00:00:00' as timestamp) )
	   and x.prem_nb = a.prem_nb
	)
)
select *
  from prem a
 where a.rn = 1
"""

In [ ]:
conn_utl = Utilities.get_utldb01p_oracle_connection()

In [ ]:
df_amda_PNs = pd.read_sql_query(
    amanda_PNs_UTL_sql, 
    conn_utl
)

In [ ]:
df_amda_PNs

In [ ]:
amda_PNs = df_amda_PNs['PREM_NB'].unique().tolist()

In [ ]:
df_1

In [ ]:
df_1[df_1['aep_premise_nb'].isin(amda_PNs)]